In [2]:
import pandas as pd
import optuna
import numpy as np

import lightgbm as lgb
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split

In [3]:
def objective(trial):
    data, target = sklearn.datasets.load_breast_cancer(return_X_y=True)
    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25)
    dtrain = lgb.Dataset(train_x, label=train_y)

    param = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    gbm = lgb.train(param, dtrain)
    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(valid_y, pred_labels)
    return accuracy

In [4]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2023-08-21 22:01:33,321] A new study created in memory with name: no-name-b41d14c5-ffee-491b-a45b-7a1cd2fd21f3
[I 2023-08-21 22:01:33,425] Trial 0 finished with value: 0.9790209790209791 and parameters: {'lambda_l1': 2.6464320750275938e-08, 'lambda_l2': 0.0005824081857676701, 'num_leaves': 92, 'feature_fraction': 0.5367186994187242, 'bagging_fraction': 0.6081234548351642, 'bagging_freq': 1, 'min_child_samples': 70}. Best is trial 0 with value: 0.9790209790209791.
[I 2023-08-21 22:01:33,471] Trial 1 finished with value: 0.958041958041958 and parameters: {'lambda_l1': 0.002565489236474182, 'lambda_l2': 4.766495647586323e-08, 'num_leaves': 15, 'feature_fraction': 0.8341942967390011, 'bagging_fraction': 0.5525144968798326, 'bagging_freq': 7, 'min_child_samples': 29}. Best is trial 0 with value: 0.9790209790209791.
[I 2023-08-21 22:01:33,514] Trial 2 finished with value: 0.993006993006993 and parameters: {'lambda_l1': 0.028878520398294357, 'lambda_l2': 2.950025455760672, 'num_leaves': 17

Number of finished trials: 100
Best trial:
  Value: 0.993006993006993
  Params: 
    lambda_l1: 0.028878520398294357
    lambda_l2: 2.950025455760672
    num_leaves: 171
    feature_fraction: 0.4837885913891835
    bagging_fraction: 0.5928728757802348
    bagging_freq: 4
    min_child_samples: 40


In [12]:
import optuna.integration.lightgbm as lgb
from lightgbm import early_stopping
from lightgbm import log_evaluation
import sklearn.datasets
from sklearn.model_selection import KFold, TimeSeriesSplit

In [13]:
data, target = sklearn.datasets.load_breast_cancer(return_X_y=True)
dtrain = lgb.Dataset(data, label=target)

params = {
    "objective": "binary",
    "metric": "binary_logloss",
    "verbosity": -1,
    "boosting_type": "gbdt",
}

tuner = lgb.LightGBMTunerCV(
    params,
    dtrain,
    folds=TimeSeriesSplit(n_splits=4),
    callbacks=[early_stopping(100), log_evaluation(100)],
)

tuner.run()

print("Best score:", tuner.best_score)
best_params = tuner.best_params
print("Best params:", best_params)
print("  Params: ")
for key, value in best_params.items():
    print("    {}: {}".format(key, value))

[I 2023-08-21 22:10:46,174] A new study created in memory with name: no-name-1362af8c-6175-47eb-92e4-6d960c111669
feature_fraction, val_score: inf:   0%|                   | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 100 rounds
[100]	cv_agg's binary_logloss: 0.142173 + 0.0971777


feature_fraction, val_score: 0.117032:  14%|8     | 1/7 [00:00<00:01,  3.05it/s]

Early stopping, best iteration is:
[59]	cv_agg's binary_logloss: 0.117032 + 0.0649477
[100]	cv_agg's binary_logloss: 0.138691 + 0.10928


feature_fraction, val_score: 0.111162:  29%|#7    | 2/7 [00:00<00:01,  3.17it/s]

...

[100]	cv_agg's binary_logloss: 0.119501 + 0.0888685
[200]	cv_agg's binary_logloss: 0.119143 + 0.0972056
Early stopping, best iteration is:
[104]	cv_agg's binary_logloss: 0.0801373 + 0.0496581


min_data_in_leaf, val_score: 0.080137:  80%|####8 | 4/5 [00:01<00:00,  3.60it/s]

[100]	cv_agg's binary_logloss: 0.251805 + 0.307742
[200]	cv_agg's binary_logloss: 0.244831 + 0.311737
Early stopping, best iteration is:
[104]	cv_agg's binary_logloss: 0.0801373 + 0.0496581
[100]	cv_agg's binary_logloss: 0.424512 + 0.323707


min_data_in_leaf, val_score: 0.080137: 100%|######| 5/5 [00:01<00:00,  3.91it/s]

[200]	cv_agg's binary_logloss: 0.421852 + 0.326334
Early stopping, best iteration is:
[104]	cv_agg's binary_logloss: 0.0801373 + 0.0496581
Best score: 0.08013725383887753
Best params: {'objective': 'binary', 'metric': 'binary_logloss', 'verbosity': -1, 'boosting_type': 'gbdt', 'feature_pre_filter': False, 'lambda_l1': 0.029673352120388748, 'lambda_l2': 4.046451579674758e-06, 'num_leaves': 16, 'feature_fraction': 0.4, 'bagging_fraction': 0.6764617799335896, 'bagging_freq': 7, 'min_child_samples': 20}
  Params: 
    objective: binary
    metric: binary_logloss
    verbosity: -1
    boosting_type: gbdt
    feature_pre_filter: False
    lambda_l1: 0.029673352120388748
    lambda_l2: 4.046451579674758e-06
    num_leaves: 16
    feature_fraction: 0.4
    bagging_fraction: 0.6764617799335896
    bagging_freq: 7
    min_child_samples: 20


In [10]:
best_params

{'objective': 'binary',
 'metric': 'binary_logloss',
 'verbosity': -1,
 'boosting_type': 'gbdt',
 'feature_pre_filter': False,
 'lambda_l1': 9.591707799735657e-06,
 'lambda_l2': 3.427943333382088e-06,
 'num_leaves': 3,
 'feature_fraction': 0.5,
 'bagging_fraction': 0.8353723328586016,
 'bagging_freq': 4,
 'min_child_samples': 25}

In [14]:
best_params

{'objective': 'binary',
 'metric': 'binary_logloss',
 'verbosity': -1,
 'boosting_type': 'gbdt',
 'feature_pre_filter': False,
 'lambda_l1': 0.029673352120388748,
 'lambda_l2': 4.046451579674758e-06,
 'num_leaves': 16,
 'feature_fraction': 0.4,
 'bagging_fraction': 0.6764617799335896,
 'bagging_freq': 7,
 'min_child_samples': 20}

In [20]:
from lightgbm import LGBMClassifier

In [43]:
trial.params

{'lambda_l1': 6.493413208962661e-07,
 'lambda_l2': 0.00048147289273148844,
 'num_leaves': 225}

In [44]:
trial = optuna_search.study_.best_trial
trial.params.items()
clf.set_params(**trial.params)

LGBMClassifier(lambda_l1=6.493413208962661e-07,
               lambda_l2=0.00048147289273148844, num_leaves=225)

In [45]:
clf

LGBMClassifier(lambda_l1=6.493413208962661e-07,
               lambda_l2=0.00048147289273148844, num_leaves=225)

In [54]:

x, y = sklearn.datasets.load_breast_cancer(return_X_y=True)


param_distributions = {
    'num_leaves': optuna.distributions.IntDistribution(2, 2*12), #31
    'max_depth': optuna.distributions.IntDistribution(1, 13), #-1
    'min_child_samples': optuna.distributions.IntDistribution(20, int(len(x) * 0.1)), #20
    #'min_child_weight': 0.001, #0.001
    #'min_split_gain': 0.0, #0.0
    
    'n_estimators': optuna.distributions.IntDistribution(100, 2000, 1), #100
    'learning_rate': optuna.distributions.FloatDistribution(0.0001, 0.3), #0.1
    
    'reg_alpha': optuna.distributions.FloatDistribution(0, 1000), #0.0
    'reg_lambda': optuna.distributions.FloatDistribution(0, 1000), #0.0
    
    'colsample_bytree': optuna.distributions.FloatDistribution(0, 1), #1.0
    
    'subsample': optuna.distributions.FloatDistribution(0, 1) , #1.0
    #'subsample_for_bin': 200000, #200000
    #'subsample_freq': 0, #0
}

clf = LGBMClassifier()

optuna_search = optuna.integration.OptunaSearchCV(
    clf, param_distributions, n_trials=100, verbose=0, cv=TimeSeriesSplit(n_splits=4),
    n_jobs=-1
)

optuna_search.fit(x, y)

print("Best trial:")
trial = optuna_search.study_.best_trial
clf.set_params(**trial.params)

clf.fit(x, y)

/var/folders/yn/9gj_99ys1mz0_tkl2kvh5wd40000gn/T/ipykernel_5710/2229661561.py:26: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = optuna.integration.OptunaSearchCV(
[I 2023-08-21 22:45:29,124] A new study created in memory with name: no-name-3fa4889e-d9c3-4a25-a34d-37bb4d11ddc5
[I 2023-08-21 22:45:33,983] Trial 6 finished with value: 0.5752212389380531 and parameters: {'num_leaves': 21, 'max_depth': 11, 'min_child_samples': 49, 'n_estimators': 219, 'learning_rate': 0.06218368572097539, 'reg_alpha': 361.88928506642173, 'reg_lambda': 57.76207655589538, 'colsample_bytree': 0.485477172781119, 'subsample': 0.9571922256008006}. Best is trial 6 with value: 0.5752212389380531.
[I 2023-08-21 22:45:34,760] Trial 11 finished with value: 0.5752212389380531 and parameters: {'num_leaves': 16, 'max_depth': 5, 'min_child_samples': 37, 'n_estimators': 254, 'learning_rate': 0.2087983793311119, 'reg_alpha': 818.789802

Best trial:


LGBMClassifier(colsample_bytree=0.5321499343393309,
               learning_rate=0.09788322077839064, max_depth=9,
               min_child_samples=48, n_estimators=816, num_leaves=24,
               reg_alpha=1.4542101216472716, reg_lambda=147.23779198895008,
               subsample=0.00024432023075557363)

In [32]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)
print(study.best_trial)

[I 2023-08-21 22:27:50,637] A new study created in memory with name: no-name-094abd0f-1b31-4cf0-bb93-e49d80d25c62
[W 2023-08-21 22:27:50,649] Trial 0 failed with parameters: {'lambda_l1': 0.00013321912215708676, 'lambda_l2': 0.04750922891171694, 'num_leaves': 168, 'feature_fraction': 0.934934889315056, 'bagging_fraction': 0.4814802657524373, 'bagging_freq': 4, 'min_child_samples': 73} because of the following error: TypeError("Trial.suggest_int() missing 2 required positional arguments: 'low' and 'high'").
Traceback (most recent call last):
  File "/Users/santiago/miniconda3/envs/thesis/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/yn/9gj_99ys1mz0_tkl2kvh5wd40000gn/T/ipykernel_5710/1730435097.py", line 17, in objective
    rf_max_depth = trial.suggest_int(param)
TypeError: Trial.suggest_int() missing 2 required positional arguments: 'low' and 'high'
[W 2023-08-21 22:27:50,650] Trial 0 failed with 

TypeError: Trial.suggest_int() missing 2 required positional arguments: 'low' and 'high'